In [0]:
from pyspark.sql import SparkSession

In [0]:
session = SparkSession.builder.appName("lr_example").getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
data = spark.read.csv("/FileStore/tables/Ecommerce_Customers.csv", inferSchema=True, header=True)

In [0]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [0]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

Out[10]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
inputCols = ['Avg Session Length', 'Time on App', 'Time on Website','Length of Membership']

assembler = VectorAssembler(inputCols=inputCols, outputCol="features")

In [0]:
output = assembler.transform(data)

In [0]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
output.head(1)

Out[20]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
final_data = output.select("features", "Yearly Amount Spent")

In [0]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [0]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                362|
|   mean| 499.63445391510135|
| stddev|  81.43252988215946|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                138|
|   mean|  498.4735276233966|
| stddev|  73.74784648479799|
|    min|  282.4712457199145|
|    max|  708.9351848669818|
+-------+-------------------+



In [0]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.554467286840293|
|-4.6363680437761445|
|  9.646690327686883|
| -4.165077678361513|
| 2.7142029998632324|
|  3.692638631582099|
| 0.4730820655724983|
| -5.611916701722066|
|-14.556277469426504|
| -6.453921455877833|
|  2.368944956839016|
|-4.8184903055983455|
| -3.675846272015633|
|  8.056368751577395|
|-10.692655042682077|
| -17.32878315222962|
|  11.80465261966998|
|-18.001780656172514|
| -4.387991937028232|
|  3.545341764848615|
+-------------------+
only showing top 20 rows



In [0]:
# Compare to target mean, stdev, min (are we close), max (are we close)
test_results.rootMeanSquaredError

Out[33]: 9.427654639399822

In [0]:
# my model explains [test_results.r2 * 100]% variance in the data
test_results.r2

Out[34]: 0.9835385850385086

In [0]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [0]:
unlabeled_data = test_data.select("features")

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8364326747734...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.3895854806643...|
|[31.5147378578019...|
|[31.5741380228732...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8512531286083...|
|[31.8648325480987...|
|[31.9048571310136...|
|[31.9262720263601...|
|[31.9563005605233...|
|[31.9673209478824...|
|[32.0123007682454...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 288.0257130067548|
|[30.8364326747734...|472.13826847076575|
|[31.1695067987115...| 417.7098404746059|
|[31.2681042107507...|427.63561085218544|
|[31.3091926408918...| 430.0065148400704|
|[31.3584771924370...| 491.4833118178933|
|[31.3895854806643...| 409.5965289944104|
|[31.5147378578019...| 495.4244046981835|
|[31.5741380228732...| 558.9655496300134|
|[31.7242025238451...| 509.8418087438383|
|[31.7366356860502...| 494.5645012986929|
|[31.7656188210424...| 501.3725719412055|
|[31.8124825597242...|396.48619125581286|
|[31.8512531286083...|  464.935877915221|
|[31.8648325480987...|450.58393551949575|
|[31.9048571310136...|491.27864057504576|
|[31.9262720263601...| 380.4002808246564|
|[31.9563005605233...| 565.1277124033713|
|[31.9673209478824...| 450.1378331766805|
|[32.0123007682454...| 489.3997113011096|
+--------------------+------------